# Window LAG

## COVID-19 Data
Notes on the data: This data was assembled based on work done by [Rodrigo Pombo](https://github.com/pomber/covid19) based on [John Hopkins University](https://systems.jhu.edu/research/public-health/ncov/), based on [World Health Organisation](https://www.who.int/health-topics/coronavirus). The data was assembled 21st April 2020 - there are no plans to keep this data set up to date.

In [1]:
import os
import pandas as pd
import findspark
os.environ['SPARK_HOME'] =  '/opt/spark'
findspark.init()

from pyspark.sql import SparkSession
sc = (SparkSession.builder.appName('app09+')
      .config('spark.sql.warehouse.dir', 'hdfs://quickstart.cloudera:8020/user/hive/warehouse')
      .config('hive.metastore.uris', 'thrift://quickstart.cloudera:9083')
      .enableHiveSupport().getOrCreate())

## Window Function
The SQL Window functions include LAG, LEAD, RANK and NTILE. These functions operate over a "window" of rows - typically these are rows in the table that are in some sense adjacent.

In [2]:
covid = sc.read.table('sqlzoo.covid')
world = sc.read.table('sqlzoo.world')

## 1. Introducing the `covid` table

The example uses a WHERE clause to show the cases in 'Italy' in March.

**Modify the query to show data from Spain**

In [3]:
from pyspark.sql.functions import *
(covid.withColumn('day', dayofmonth(covid['whn']))
    .filter((covid['name']=='Spain') & (month(covid['whn'])==3))
    .select('name', 'day', 'confirmed', 'deaths', 'recovered')
    .orderBy('day')
    .toPandas())

,name,day,confirmed,deaths,recovered
0,Spain,1,84,0,2
1,Spain,2,120,0,2
2,Spain,3,165,1,2
3,Spain,4,222,2,2
4,Spain,5,259,3,2
5,Spain,6,400,5,2
6,Spain,7,500,10,30
7,Spain,8,673,17,30
8,Spain,9,1073,28,32
9,Spain,10,1695,35,32


## 2. Introducing the LAG function

The LAG function is used to show data from the preceding row or the table. When lining up rows the data is partitioned by country name and ordered by the data whn. That means that only data from Italy is considered.

**Modify the query to show confirmed for the day before.**

In [4]:
from pyspark.sql import Window
(covid.withColumn('day', dayofmonth(covid['whn']))
     .withColumn('mo', month(covid['whn']))
     .filter((covid['name']=='Italy') & (month(covid['whn'])==3))
     .withColumn('lag_cfrm', lag(col('confirmed')).over(Window.orderBy('day')))
     .select('name', 'day', 'confirmed', 'mo', 'lag_cfrm')
     .toPandas())

,name,day,confirmed,mo,lag_cfrm
0,Italy,1,1694,3,NaN
1,Italy,2,2036,3,1694.0
2,Italy,3,2502,3,2036.0
3,Italy,4,3089,3,2502.0
4,Italy,5,3858,3,3089.0
5,Italy,6,4636,3,3858.0
6,Italy,7,5883,3,4636.0
7,Italy,8,7375,3,5883.0
8,Italy,9,9172,3,7375.0
9,Italy,10,10149,3,9172.0


### LAG operation

Here is the correct query showing the cases for the day before:

```sql
SELECT name, DAY(whn), confirmed,
   LAG(confirmed, 1) OVER (partition by name ORDER BY whn) AS lag
 FROM covid
WHERE name = 'Italy'
AND MONTH(whn) = 3
ORDER BY whn
```

Notice how the values in the LAG column match the value of the row diagonally above and to the left.

name | DAY(whn) | confirmed | dbf
------|---|------|----------
Italy | 1 | **1694** | null
Italy | 2 | 2036 | **1694**
Italy | 3 | 2502 | 2036
Italy | 4 | 3089 | 2502
Italy | 5 | **3858** | 3089
Italy | 6 | 4636 | **3858**
Italy | 7 | 5883 | 4636
Italy | 8 | 7375 | 5883
Italy | 9 | 9172 | 7375
Italy | 10 | 10149 | 9172
... | | |

## 3. Number of new cases

The number of confirmed case is cumulative - but we can use LAG to recover the number of new cases reported for each day.

**Show the number of new cases for each day, for Italy, for March.**

In [5]:
(covid.withColumn('day', dayofmonth(covid['whn']))
     .filter((covid['name']=='Italy') & (month(covid['whn'])==3))
     .withColumn('new', col('confirmed') - lag(col('confirmed')).over(
         Window.orderBy('day')))
     .select('name', 'day', 'new')
     .toPandas())

,name,day,new
0,Italy,1,NaN
1,Italy,2,342.0
2,Italy,3,466.0
3,Italy,4,587.0
4,Italy,5,769.0
5,Italy,6,778.0
6,Italy,7,1247.0
7,Italy,8,1492.0
8,Italy,9,1797.0
9,Italy,10,977.0


## 4. Weekly changes

The data gathered are necessarily estimates and are inaccurate. However by taking a longer time span we can mitigate some of the effects.

You can filter the data to view only Monday's figures **WHERE WEEKDAY(whn) = 0**.

**Show the number of new cases in Italy for each week - show Monday only.**

In [6]:
(covid.filter((covid['name']=='Italy') & (dayofweek(covid['whn'])==1))
     .withColumn('new', col('confirmed')-lag(col('confirmed')).over(Window.orderBy('whn')))
     .select('name', 'whn', 'new')
     .toPandas()
)

,name,whn,new
0,Italy,2020-01-26,NaN
1,Italy,2020-02-02,2.0
2,Italy,2020-02-09,1.0
3,Italy,2020-02-16,0.0
4,Italy,2020-02-23,152.0
5,Italy,2020-03-01,1539.0
6,Italy,2020-03-08,5681.0
7,Italy,2020-03-15,17372.0
8,Italy,2020-03-22,34391.0
9,Italy,2020-03-29,38551.0


## 5. LAG using a JOIN

You can JOIN a table using DATE arithmetic. This will give different results if data is missing.

**Show the number of new cases in Italy for each week - show Monday only.**

In the sample query we JOIN this week tw with last week lw using the DATE_ADD function.

In [7]:
a = covid.filter((covid['name']=='Italy') & (dayofweek(covid['whn'])==1))
(a.select('whn', 'name', 'confirmed')
     .join(a
           .withColumn('whn', date_add(a['whn'], 7))
           .withColumnRenamed('confirmed', 'confirmed2')
           .select('whn', 'name', 'confirmed2'), 
           ['whn', 'name'], how='left')
     .withColumn('new', col('confirmed')-col('confirmed2'))
     .select('name', 'whn', 'new')
     .toPandas())

,name,whn,new
0,Italy,2020-01-26,NaN
1,Italy,2020-02-02,2.0
2,Italy,2020-02-09,1.0
3,Italy,2020-02-16,0.0
4,Italy,2020-02-23,152.0
5,Italy,2020-03-01,1539.0
6,Italy,2020-03-08,5681.0
7,Italy,2020-03-15,17372.0
8,Italy,2020-03-22,34391.0
9,Italy,2020-03-29,38551.0


## 6. RANK()

The query shown shows the number of confirmed cases together with the world ranking for cases.

United States has the highest number, Spain is number 2...

Notice that while Spain has the second highest confirmed cases, Italy has the second highest number of deaths due to the virus.

**Include the ranking for the number of deaths in the table. Only include countries with a population of at least 10 million.**

In [8]:
a = covid.join(world.select('name', 'population'), 'name')
(a.withColumn('rc1', rank().over(Window.orderBy(col('confirmed').desc())))
     .withColumn('rc2', rank().over(Window.orderBy(col('deaths').desc())))
    .filter((col('whn')=='2020-04-20') & (col('population')>=1e7))
    .select('name', 'confirmed', 'rc1', 'deaths', 'rc2')
    .orderBy(col('confirmed').desc())
    .toPandas())

,name,confirmed,rc1,deaths,rc2
0,United States,784326,1,42094,1
1,Spain,200210,21,20852,17
2,Italy,181228,27,24114,8
3,France,156480,42,20292,21
4,Germany,147065,52,4862,119
...,...,...,...,...,...
85,Angola,24,5317,2,3508
86,Malawi,17,5680,2,3508
87,Burundi,5,7012,1,3887
88,South Sudan,4,7170,0,4763


## 7. Infection rate

The query shown includes a JOIN t the world table so we can access the total population of each country and calculate infection rates (in cases per 100,000).

**Show the infect rate ranking for each country. Only include countries with a population of at least 10 million.**

In [9]:
# a was obtained in #6
b = a.filter((a['whn']=='2020-04-20') & (a['population']>1e7))
(b.withColumn('r_inf', round(1e5*b['confirmed']/b['population']))
     .withColumn('rc', rank().over(Window.orderBy(col('r_inf'))))
     .orderBy(col('population').desc())
     .select('name', 'r_inf', 'rc')
     .toPandas())

,name,r_inf,rc
0,China,6.0,50
1,India,1.0,23
2,United States,238.0,87
3,Indonesia,3.0,35
4,Pakistan,4.0,41
...,...,...,...
85,Jordan,4.0,41
86,Dominican Republic,48.0,73
87,Sweden,143.0,81
88,Portugal,203.0,85


## 8. Turning the corner

For each country that has had at last 1000 new cases in a single day, show the date of the peak number of new cases.

In [10]:
(covid.withColumn('new', (covid['confirmed']-lag(col('confirmed')).over(
    Window.partitionBy('name').orderBy('whn'))))
     .fillna({'new': 0})
     .withColumn('rc', rank().over(Window.partitionBy('name').orderBy(col('new').desc())))
     .filter((col('rc')==1) & (col('new')>1000))
     .select('name', 'whn', 'new')
     .orderBy('whn', 'new')
     .toPandas())

,name,whn,new
0,China,2020-02-13,15136
1,Italy,2020-03-21,6557
2,Switzerland,2020-03-23,1321
3,Israel,2020-03-25,1131
4,Spain,2020-03-25,9630
5,Austria,2020-03-26,1321
6,Germany,2020-03-27,6933
7,Iran,2020-03-30,3186
8,Canada,2020-04-05,2778
9,Netherlands,2020-04-10,1346


In [11]:
sc.stop()